In [1]:
import os

os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import create_directories, read_yaml

In [9]:
from importlib.resources import path


class ConfigurationManager:
    def __init__(
        self, config_file_path=CONFIG_FILE_PATH, param_file_path=PARAMS_FILE_PATH
    ) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), model_ckpt_dir])
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        return prepare_callback_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile

import tensorflow as tf

In [13]:
import time


class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_log_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath , 
            save_best_only=True
        )

    def get_tb_ckpt_callback(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

In [15]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    prepare_callbacks.get_tb_ckpt_callback()
except Exception as e:
    raise e

2022-09-21 23:58:20.404 | INFO     | deepClassifier.utils.common:read_yaml:30 - yaml file: configs/config.yaml loaded successfully
[deepClassifier.utils.common] yaml file: configs/config.yaml loaded successfully
2022-09-21 23:58:20.406 | INFO     | deepClassifier.utils.common:read_yaml:30 - yaml file: params.yaml loaded successfully
[deepClassifier.utils.common] yaml file: params.yaml loaded successfully
2022-09-21 23:58:20.407 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts
[deepClassifier.utils.common] created directory at: artifacts
2022-09-21 23:58:20.409 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[deepClassifier.utils.common] created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
2022-09-21 23:58:20.410 | INFO     | deepClassifier.utils.common:create_directories:49 - created directory at: artifacts/prepare_callbacks/checkpoint_